# API request and csv appender for IQAir data

In [1]:
import pandas as pd
import requests
import json
import csv
from datetime import datetime, timedelta
import numpy as np
import matplotlib.dates as mdates
from matplotlib.dates import DateFormatter

Notes on data:
- The PM2.5 number is actually the "Now Cast" (https://en.wikipedia.org/wiki/NowCast_(air_quality_index)), which is calculated based on the last 12 hours of PM2.5 concentrations, weighting recent concentrations more heavily. It is not really possible to back-calculate PM2.5 concentration from Now Cast values because there are so many inputs (12 hours' worth).
- Source on this data: https://aqicn.org/faq/2015-03-15/air-quality-nowcast-a-beginners-guide/

In [2]:
# API pulls from https://aqicn.org/city/utah/salt-lake-city
response = requests.get(f'http://api.waqi.info/feed/@4469/?token=dae62b01eb4dfedf1f19937d2c3bedef664c4805')
print(response.status_code)

200


In [3]:
data = response.text

In [4]:
json_response = json.loads(data)

In [5]:
json_response

{'status': 'ok',
 'data': {'aqi': 18,
  'idx': 4469,
  'attributions': [{'url': 'https://deq.utah.gov/division-air-quality',
    'name': 'UTAH department of environmental quality',
    'logo': 'US-UtahDEQ.png'},
   {'url': 'https://waqi.info/', 'name': 'World Air Quality Index Project'}],
  'city': {'geo': [40.733501, -111.871696],
   'name': 'Salt Lake City, Utah',
   'url': 'https://aqicn.org/city/utah/salt-lake-city',
   'location': ''},
  'dominentpol': 'o3',
  'iaqi': {'co': {'v': 2.9},
   'h': {'v': 88},
   'no2': {'v': 14.9},
   'o3': {'v': 17.6},
   'p': {'v': 1018.9},
   'pm25': {'v': 16},
   't': {'v': 31.1111},
   'w': {'v': 0.7},
   'wg': {'v': 3.7}},
  'time': {'s': '2025-12-05 17:00:00',
   'tz': '-07:00',
   'v': 1764954000,
   'iso': '2025-12-05T17:00:00-07:00'},
  'forecast': {'daily': {'pm10': [{'avg': 21,
      'day': '2025-12-03',
      'max': 22,
      'min': 17},
     {'avg': 20, 'day': '2025-12-04', 'max': 25, 'min': 15},
     {'avg': 10, 'day': '2025-12-05', 'ma

In [6]:
json_data = json_response['data']
json_data

{'aqi': 18,
 'idx': 4469,
 'attributions': [{'url': 'https://deq.utah.gov/division-air-quality',
   'name': 'UTAH department of environmental quality',
   'logo': 'US-UtahDEQ.png'},
  {'url': 'https://waqi.info/', 'name': 'World Air Quality Index Project'}],
 'city': {'geo': [40.733501, -111.871696],
  'name': 'Salt Lake City, Utah',
  'url': 'https://aqicn.org/city/utah/salt-lake-city',
  'location': ''},
 'dominentpol': 'o3',
 'iaqi': {'co': {'v': 2.9},
  'h': {'v': 88},
  'no2': {'v': 14.9},
  'o3': {'v': 17.6},
  'p': {'v': 1018.9},
  'pm25': {'v': 16},
  't': {'v': 31.1111},
  'w': {'v': 0.7},
  'wg': {'v': 3.7}},
 'time': {'s': '2025-12-05 17:00:00',
  'tz': '-07:00',
  'v': 1764954000,
  'iso': '2025-12-05T17:00:00-07:00'},
 'forecast': {'daily': {'pm10': [{'avg': 21,
     'day': '2025-12-03',
     'max': 22,
     'min': 17},
    {'avg': 20, 'day': '2025-12-04', 'max': 25, 'min': 15},
    {'avg': 10, 'day': '2025-12-05', 'max': 17, 'min': 8},
    {'avg': 7, 'day': '2025-12-06', 

In [7]:
json_loc = json_data['city']
json_loc

{'geo': [40.733501, -111.871696],
 'name': 'Salt Lake City, Utah',
 'url': 'https://aqicn.org/city/utah/salt-lake-city',
 'location': ''}

In [8]:
json_aqi = json_data['iaqi']
json_aqi

{'co': {'v': 2.9},
 'h': {'v': 88},
 'no2': {'v': 14.9},
 'o3': {'v': 17.6},
 'p': {'v': 1018.9},
 'pm25': {'v': 16},
 't': {'v': 31.1111},
 'w': {'v': 0.7},
 'wg': {'v': 3.7}}

In [9]:
json_timestamp = json_data['time']
json_timestamp

{'s': '2025-12-05 17:00:00',
 'tz': '-07:00',
 'v': 1764954000,
 'iso': '2025-12-05T17:00:00-07:00'}

In [10]:
values = []
headers = []

for i in json_data['iaqi']:
    headers.append(i)
    values.append(json_aqi[i]['v'])

values

[2.9, 88, 14.9, 17.6, 1018.9, 16, 31.1111, 0.7, 3.7]

In [11]:
headers

['co', 'h', 'no2', 'o3', 'p', 'pm25', 't', 'w', 'wg']

In [12]:
df = pd.DataFrame(values).T
df

,0,1,2,3,4,5,6,7,8
0,2.9,88.0,14.9,17.6,1018.9,16.0,31.1111,0.7,3.7


In [13]:
df.columns = headers
df

,co,h,no2,o3,p,pm25,t,w,wg
0,2.9,88.0,14.9,17.6,1018.9,16.0,31.1111,0.7,3.7


In [14]:
date_string = json_data['time']['s']
date_string

'2025-12-05 17:00:00'

In [15]:
format_string = "%Y-%m-%d %H:%M:%S"
datetime_object = datetime.strptime(date_string, format_string)
datetime_object

datetime.datetime(2025, 12, 5, 17, 0)

In [16]:
#Convert to MST
if json_data['time']['tz'] == '-06:00':
    datetime_object = datetime_object - timedelta(hours=1)
datetime_object

datetime.datetime(2025, 12, 5, 17, 0)

In [17]:
#Convert timestamp back to string
# datetime_string = datetime_object.strftime('%Y-%m-%d %H:%M:%S')
datetime_string = datetime_object.strftime('%m/%d/%Y %H:%M')
datetime_string

'12/05/2025 17:00'

In [18]:
datetime_list = [datetime_string]
datetime_list

['12/05/2025 17:00']

In [19]:
cols = df.columns.tolist()
cols.insert(0, 'timestamp')
df['timestamp'] = datetime_list
df = df[cols]
df

,timestamp,co,h,no2,o3,p,pm25,t,w,wg
0,12/05/2025 17:00,2.9,88.0,14.9,17.6,1018.9,16.0,31.1111,0.7,3.7


In [20]:
new_data = df.iloc[0].tolist()
new_data

['12/05/2025 17:00', 2.9, 88.0, 14.9, 17.6, 1018.9, 16.0, 31.1111, 0.7, 3.7]

In [21]:
def append_to_csv(file_path, data_row):
    """
    Appends a single row of data to an existing CSV file.

    Args:
        file_path (str): The path to the CSV file.
        data_row (list): A list representing the row of data to append.
    """
    try:
        with open(file_path, 'a', newline='') as csvfile:
            writer = csv.writer(csvfile)
            writer.writerow(data_row)
        print(f"Data '{data_row}' appended successfully to '{file_path}'.")
    except FileNotFoundError:
        print(f"Error: The file '{file_path}' was not found.")
    except Exception as e:
        print(f"An error occurred: {e}")

In [25]:
csv_file = 'AirQualityOpenData_API_dump.csv'

#Check that this data call is new data by checking last timestamp
df_existingdata = pd.read_csv(csv_file)
print(df_existingdata.head())

          timestamp   co   h  no2    o3       p  pm25 (Now Cast, not ug/m3)  \
0  10/15/2025 15:00  1.8  50  2.8  36.8  1006.7                           6   
1  10/15/2025 16:00  1.8  51  2.8  36.1  1006.9                           8   
2  10/16/2025 14:00  1.2  69  4.7  31.2  1015.1                          13   
3  10/16/2025 15:00  1.2  65  4.7  32.0  1015.2                          18   
4  10/16/2025 18:00  2.8  58  8.4  28.9  1015.6                          16   

         t    w   wg  
0  10.5556  1.5  6.0  
1  10.5556  1.6  5.6  
2  20.5556  1.6  0.5  
3  18.3333  1.6  2.0  
4  14.4444  1.6  1.7  


In [26]:
print(df_existingdata.tail(1))

           timestamp   co   h   no2   o3       p  pm25 (Now Cast, not ug/m3)  \
977  12/4/2025 12:00  9.2  70  29.7  0.8  1027.8                          55   

           t    w   wg  
977  3.91111  1.5  0.7  


In [27]:
last_timestamp = df_existingdata.iloc[-1,0]
print(last_timestamp)

12/4/2025 12:00


In [28]:
#Turn last timestamp into datetime object
# format_string = "%Y-%m-%d %H:%M:%S"
format_string = "%m/%d/%Y %H:%M"
last_timestamp_object = datetime.strptime(last_timestamp, format_string)
print(last_timestamp_object)

2025-12-04 12:00:00


In [29]:
#If this hour's data is not yet appended to the csv file, append data.
if last_timestamp_object < datetime_object:
    append_to_csv(csv_file,new_data)
else:
    print('last_timestamp_object is not less than current timestamp')

Data '['12/05/2025 17:00', 2.9, 88.0, 14.9, 17.6, 1018.9, 16.0, 31.1111, 0.7, 3.7]' appended successfully to 'AirQualityOpenData_API_dump.csv'.
